# LangChain: Fundamental

Langchain is a framework for building applications with languange models. In this tutorial, we will learn the fundamental concepts of Langchain. 

1. Benefits of Langchain
2. Basic Runnables in Langchain: Chatmodel, Prompt, Message

The benefits of LangChain are:

* Easy switch between LLM providers: LangChain provides a unified interface for different LLM providers. Langchain exposes a standard interface for key components such as models, prompts, tool calling, output parsers,and chains.Therefore, it is easy for developers to switch between providers. It is mainly supported by langchain-core and langchain-community.

* Easy to build applications with language models: combine multiple components and models into more complex applications, there’s a growing need to efficiently connect these elements into control flows that can accomplish diverse tasks. Orchestration is crucial for building such applications. It is mainly supported by langgraph.

* Easy to track and debug: LangChain provides a platform for observability and evaluations. It is mainly supported by langsmith.

### OpenAI API

To run the following OpenAI model, you need to have the OpenAI API key. Here, we use the [environment variable](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety) `OPENAI_API_KEY` to store the API key. You can also set the API key in the code directly.

```
import os
os.environ["OPENAI_API_KEY"] = "your_api_key"
```


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}")
])
llm = ChatOpenAI(model='gpt-4o-mini')
chain = prompt | llm
result = chain.invoke({"input": "Hello, what is the capital of Canada?"})
print(result.content)


The capital of Canada is Ottawa.


### Ollama

To run the following code, you need to install Ollama and run the ollama locally. In the terminal, run the following command to start the ollama server:

```
ollama run qwen2.5:3b
```


In [2]:
from langchain_ollama import ChatOllama
llm_qwen = ChatOllama(model="qwen2.5:3b")
chain_qwen = prompt | llm_qwen
result_qwen = chain_qwen.invoke({"input": "Hello, what is the capital of Canada?"})
print(result_qwen.content)

The capital of Canada is Ottawa.



### HuggingFace

To run the following code, you need to install HuggingFace and run the HuggingFace server.

In [5]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
llm_hf = HuggingFacePipeline.from_model_id(
    model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task="text-generation",
    pipeline_kwargs=dict(max_new_tokens=256, 
                         do_sample=True, 
                         temperature=0.7, 
                         top_k=50, 
                         top_p=0.95,
    ),
)
chat_model = ChatHuggingFace(llm=llm_hf)
chain_hf = prompt | chat_model.bind(skip_prompt=True)
result_hf = chain_hf.invoke({"input": "Hello, what is the capital of Canada?"})
print(result_hf.content)

Device set to use mps:0


The capital of Canada is Ottawa, located in the province of Ontario.


In [2]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
llm_hf = HuggingFacePipeline.from_model_id(
    model_id="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    task="text-generation",
    pipeline_kwargs=dict(max_new_tokens=256, 
                         do_sample=True, 
                         temperature=0.7, 
                         top_k=50, 
                         top_p=0.95,
    ),
)
chat_model = ChatHuggingFace(llm=llm_hf)

Device set to use mps:0


LangChain’s core abstraction is the *Runnable* interface, which provides a standard way to compose and execute language model chains. Via LCEL, runnables can be chained together using operators like (pipe) and + (combine), allowing you to build complex workflows from simple components. They support batch processing, streaming, async execution and other advanced features. The supported interfaces are as follows:
| Interface | Description |
| --- | --- |
| Invoked | A single input is transformed into an output |
| Batched | Multiple inputs are efficiently transformed into outputs |
| Streamed | Outputs are streamed as they are produced |
| Inspected | Schematic information about Runnable's input, output, and configuration can be accessed |
| Composed | Multiple Runnables can be composed to work together using the LangChain Expression Language (LCEL) to create complex pipelines |

And the major types of predefined runnables are as follows:

| Component | Input Type | Output Type |
| --- | --- | --- |
| Prompt | dictionary | PromptValue |
| ChatModel | a string, list of chat messages or a PromptValue | ChatMessage |
| LLM | a string, list of chat messages or a PromptValue | String |
| OutputParser | the output of an LLM or ChatModel | Depends on the parser |
| Retriever | a string | List of Documents |
| Tool | a string or dictionary, depending on the tool | Depends on the tool |


### ChatModel & LLM 

ChatModel is a wrapper around an LLM which provide a standardized way to interact with modern LLMs through a message-based interface. It should be preferred for development than Legacy LLM.

In [9]:
from langchain_openai import OpenAI
llm = OpenAI()
response = llm.invoke("What is after Monday?")
print(response)



Tuesday.


In [10]:

from langchain_openai import ChatOpenAI
chat = ChatOpenAI()
response = chat.invoke("What is after Monday?")
print(response)
# Returns: AIMessage(content='Tuesday')

content='Tuesday' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 12, 'total_tokens': 14, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4c0d7a68-6458-4636-aa1a-cc1ba7a8addc-0' usage_metadata={'input_tokens': 12, 'output_tokens': 2, 'total_tokens': 14, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


#### Prompt & Message

1. Messages are the fundamental units of communication in chat models. They represent individual pieces of a conversation and have specific roles and content. 
2. Prompts are templates that help structure how we format inputs before sending them to language models.

For message, it has four roles: Human, System, AI, and ToolMessage. 

HumanMessage: User inputs

SystemMessage: Sets behavior/context for the AI

AIMessage: Model responses

ToolMessage: Results from tool calls

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage
   
messages = [
       SystemMessage(content="You are a helpful assistant"),
       HumanMessage(content="What is LangChain?")
   ]

Messages can help us to directly interact with ChatModels with the fine-grained control over conversion and specific conversion roles. And prompts can be beneficial for reusable templates, standardization inputs with variable contents.

In [17]:
from langchain_core.prompts import ChatPromptTemplate
chat_template = ChatPromptTemplate([
       ("system", "You are a helpful assistant"),
       ("user", "Tell me about {topic}")
])

In [18]:
print(chat_template.invoke({"topic": "AI"}))

messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about AI', additional_kwargs={}, response_metadata={})]


In the above, we can see string is used to format the message. And we can also pass a list of messages to format the prompt. The example could be found as below:

In [20]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])
prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

The following code is an example of how to use the prompt and message in LangChain to create the few-shot prompts.

In [22]:
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)
examples = [
    {"input": "what is 2~2", "output": "4"},
    {"input": "what is 2~3", "output": "6"},
    {"input": "what is 4~9", "output": "36"},
    {"input": "what is 25~2", "output": "50"},
]
example_prompt = ChatPromptTemplate(
    [('human', '{input}'), ('ai', '{output}')]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    # This is a prompt template used to format each individual example.
    example_prompt=example_prompt,
)
final_prompt = ChatPromptTemplate(
    [
        ('system', 'You are a helpful AI Assistant and you should use the examples to answer the question'),
        few_shot_prompt,
        ('human', '{input}'),
    ]
)
print(final_prompt)
chain = final_prompt | llm
chain.invoke({"input": "what is 4~4?"})

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful AI Assistant and you should use the examples to answer the question'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': 'what is 2~2', 'output': '4'}, {'input': 'what is 2~3', 'output': '6'}, {'input': 'what is 4~9', 'output': '36'}, {'input': 'what is 25~2', 'output': '50'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs=

'\nAI: 16'